<div style="width: 98%; max-width: 1400px; margin: 0 auto; font-family: 'Segoe UI', Helvetica, Arial, sans-serif;">

<div style="
        background: linear-gradient(135deg, #667eea 0%, #764ba2 50%, #f093fb 100%);
        border-radius: 12px;
        padding: 35px;
        text-align: center;
        margin-bottom: 25px;
        border: 2px solid #667eea;
        box-shadow: 0 8px 20px rgba(102, 126, 234, 0.4);
    ">
        <h1 style="
            color: #fff;
            margin: 0;
            font-size: 32px;
            font-weight: 300;
            letter-spacing: 4px;
            text-transform: uppercase;
        ">
            ◈ Analysis & Visualization Tools
        </h1>
        <h2 style="
            color: #f0f0f0;
            margin: 15px 0 0 0;
            font-weight: 300;
            font-style: italic;
            font-size: 18px;
        ">
            Interactive Diagnostics & Exploration Suite for LAMOST DR5
        </h2>
        <div style="
            margin-top: 20px;
            padding-top: 15px;
            border-top: 1px solid rgba(255, 255, 255, 0.3);
        ">
            <span style="color: #f0f0f0; font-size: 13px; text-transform: uppercase; letter-spacing: 1px;">
                11 Interactive Tools • Real-time Diagnostics • Model Interpretability
            </span>
        </div>
    </div>

<div style="
        background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
        border: 2px solid #667eea;
        border-radius: 12px;
        padding: 25px;
        margin-bottom: 20px;
        box-shadow: 0 4px 15px rgba(0,0,0,0.3);
    ">
        <h4 style="
            color: #667eea;
            margin: 0 0 18px 0;
            font-size: 16px;
            text-transform: uppercase;
            letter-spacing: 1px;
            text-align: center;
            border-bottom: 2px solid rgba(102, 126, 234, 0.4);
            padding-bottom: 8px;
        ">◈ Notebook Purpose & Scope</h4>
        
<p style="color: #e0e0e0; font-size: 15px; line-height: 1.8; margin: 15px 0;">
            This notebook provides a comprehensive suite of <strong>11 interactive diagnostic tools</strong> for exploring, analyzing, and debugging LAMOST DR5 spectral data without requiring full pipeline re-execution. Each tool is designed for specific analysis tasks, from basic data quality checks to advanced model interpretability via SHAP analysis.
        </p>
        
<div style="background: rgba(102, 126, 234, 0.15); padding: 15px; border-radius: 8px; margin-top: 15px;">
            <strong style="color: #667eea; font-size: 14px;">Key Features :</strong>
            <ul style="color: #c0c0c0; margin: 10px 0; padding-left: 20px; line-height: 1.8;">
                <li>Widget-based interactive interfaces for all tools</li>
                <li>Automatic logging and artifact saving</li>
                <li>Real-time visualizations with publication-ready output</li>
                <li>Modular design — use tools independently or in sequence</li>
            </ul>
        </div>
    </div>

</div>

<a id="toc"></a>

<h2 id="interactive-table-of-contents">Interactive Table of Contents</h2>

<div style="
    width: 100%;
    box-sizing: border-box;
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    padding: 25px;
    border-radius: 12px;
    border: 1px solid #667eea;
    color: white;
">

<div style="display: flex; flex-wrap: wrap; gap: 20px;">

<div style="
            flex: 1 1 100%; 
            background: rgba(102, 126, 234, 0.15); 
            padding: 20px; 
            border-radius: 8px; 
            border-left: 4px solid #667eea; 
            box-sizing: border-box;
        ">
            <strong style="color: #667eea; font-size: 16px; display: block; margin-bottom: 10px;">◈ Setup & Configuration</strong>
            <ul style="margin: 0; padding-left: 0; list-style: none; line-height: 1.8;">
                <li><span style="color: #667eea; margin-right: 8px;">→</span><a href="#setup" style="color: #a0a0ff; text-decoration: none;">Environment Setup & Imports</a></li>
            </ul>
        </div>

<div style="
            flex: 1 1 45%; 
            min-width: 300px; 
            background: rgba(46, 204, 113, 0.15); 
            padding: 20px; 
            border-radius: 8px; 
            border-left: 4px solid #2ECC71; 
            box-sizing: border-box;
        ">
            <strong style="color: #2ECC71; font-size: 16px; display: block; margin-bottom: 10px;">◈ Dataset Analysis</strong>
            <ul style="margin: 0; padding-left: 0; list-style: none; line-height: 1.8; font-size: 14px;">
                <li><span style="color: #2ECC71; margin-right: 8px;">→</span><a href="#dashboard" style="color: #82E0AA; text-decoration: none;">Dataset Dashboard</a></li>
                <li><span style="color: #2ECC71; margin-right: 8px;">→</span><a href="#nulls" style="color: #82E0AA; text-decoration: none;">Null Features Analysis</a></li>
                <li><span style="color: #2ECC71; margin-right: 8px;">→</span><a href="#feature-explorer" style="color: #82E0AA; text-decoration: none;">Feature Explorer</a></li>
                <li><span style="color: #2ECC71; margin-right: 8px;">→</span><a href="#subclasses" style="color: #82E0AA; text-decoration: none;">Subclass Distribution</a></li>
            </ul>
        </div>

<div style="
            flex: 1 1 45%; 
            min-width: 300px; 
            background: rgba(52, 152, 219, 0.15); 
            padding: 20px; 
            border-radius: 8px; 
            border-left: 4px solid #3498DB; 
            box-sizing: border-box;
        ">
            <strong style="color: #3498DB; font-size: 16px; display: block; margin-bottom: 10px;">◈ Spectral Tools</strong>
            <ul style="margin: 0; padding-left: 0; list-style: none; line-height: 1.8; font-size: 14px;">
                <li><span style="color: #3498DB; margin-right: 8px;">→</span><a href="#fits" style="color: #85C1E9; text-decoration: none;">FITS Header Explorer</a></li>
                <li><span style="color: #3498DB; margin-right: 8px;">→</span><a href="#tuning" style="color: #85C1E9; text-decoration: none;">Peak Detection Tuner</a></li>
                <li><span style="color: #3498DB; margin-right: 8px;">→</span><a href="#compare-norm" style="color: #85C1E9; text-decoration: none;">Normalization Comparator</a></li>
                <li><span style="color: #3498DB; margin-right: 8px;">→</span><a href="#compare-spectra" style="color: #85C1E9; text-decoration: none;">Spectra Comparator</a></li>
            </ul>
        </div>

<div style="
            flex: 1 1 45%; 
            min-width: 300px; 
            background: rgba(155, 89, 182, 0.15); 
            padding: 20px; 
            border-radius: 8px; 
            border-left: 4px solid #9B59B6; 
            box-sizing: border-box;
        ">
            <strong style="color: #9B59B6; font-size: 16px; display: block; margin-bottom: 10px;">◈ Spatial Analysis</strong>
            <ul style="margin: 0; padding-left: 0; list-style: none; line-height: 1.8; font-size: 14px;">
                <li><span style="color: #9B59B6; margin-right: 8px;">→</span><a href="#sky" style="color: #C39BD3; text-decoration: none;">Sky Coverage Map</a></li>
            </ul>
        </div>

<div style="
            flex: 1 1 45%; 
            min-width: 300px; 
            background: rgba(230, 126, 34, 0.15); 
            padding: 20px; 
            border-radius: 8px; 
            border-left: 4px solid #E67E22; 
            box-sizing: border-box;
        ">
            <strong style="color: #E67E22; font-size: 16px; display: block; margin-bottom: 10px;">◈ Model Analysis</strong>
            <ul style="margin: 0; padding-left: 0; list-style: none; line-height: 1.8; font-size: 14px;">
                <li><span style="color: #E67E22; margin-right: 8px;">→</span><a href="#inspector" style="color: #F5B041; text-decoration: none;">Model Inspector</a></li>
                <li><span style="color: #E67E22; margin-right: 8px;">→</span><a href="#shap" style="color: #F5B041; text-decoration: none;">SHAP Interpretability</a></li>
            </ul>
        </div>

</div>
</div>

<div style="height: 40px;"></div>

<hr style="
    border: 0;
    height: 3px;
    background: linear-gradient(90deg, transparent, #667eea, transparent);
    border-radius: 2px;
">

<div style="height: 40px;"></div>

<a id="setup"></a>

# Setup & Configuration

<div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
    <a href="#toc" style="color: #667eea; text-decoration: none; font-size: 12px;">↩ Back to TOC</a>
</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #667eea;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Initialize the working environment by detecting the project root, configuring system paths, and loading the `AstroVisualizer` interface.

**What Happens :**
- Automatic project root detection
- Addition of `src/` to `sys.path` for module imports
- Loading of `setup_project_env()` utility (paths & constants)
- Instantiation of `AstroVisualizer` with all 11 interactive tools

**Expected Output :**  
Success message : *"Setup complete. Visualization tools are ready."* and a ready-to-use `visualizer` object.

**Components Loaded :**
- **AstroVisualizer** : Main interface providing access to all diagnostic tools
- **Project Paths** : RAW_DATA_DIR, CATALOG_DIR, PROCESSED_DIR, MODELS_DIR, REPORTS_DIR
- **Pipeline Utilities** : For model reloading and artifact management

</div>

In [ ]:
# ============================================================================
# ENVIRONMENT SETUP — AstroVisualizer Initialization
# ============================================================================
"""
This cell initializes the complete visualization and diagnostics environment.

Components:
- AstroVisualizer: Main interface for 11 interactive tools
- setup_project_env(): Path configuration utility
- Automatic sys.path configuration for module imports

Requirements:
- Valid project structure (data/, models/, logs/ directories)
- Python 3.11+ with required dependencies installed
"""

# External library imports
import os

# AstroSpectro module imports
from utils import setup_project_env
from tools.visualizer import AstroVisualizer

# Environment initialization
print("Initializing visualization environment...")
paths = setup_project_env()

# Instantiate AstroVisualizer with project paths
print("Loading AstroVisualizer tools...")
visualizer = AstroVisualizer(paths)

# Confirmation
print("\n" + "═" * 70)
print("   SETUP COMPLETE — VISUALIZATION TOOLS READY")
print("═" * 70)
print("   Available Tools  : 11 interactive diagnostic interfaces")
print("   Visualizer Status: Active")
print("═" * 70)

<div style="height: 50px;"></div>

<hr style="
    border: 0;
    height: 3px;
    background: linear-gradient(90deg, transparent, #2ECC71, transparent);
    border-radius: 2px;
">

<div style="height: 50px;"></div>

<div style="
    background: linear-gradient(135deg, #2ECC71 0%, #27AE60 100%);
    padding: 15px 25px;
    border-radius: 10px;
    margin: 30px 0;
    text-align: center;
    border: 2px solid #2ECC71;
">
    <h2 style="color: white; margin: 0; font-size: 24px; font-weight: 300; letter-spacing: 2px;">
        ◆ DATASET ANALYSIS TOOLS
    </h2>
</div>

<a id="dashboard"></a>

## Tool 1 : Dataset Dashboard

<div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
    <a href="#toc" style="color: #2ECC71; text-decoration: none; font-size: 12px;">↩ Back to TOC</a>
</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #2ECC71;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Real-time dashboard providing a comprehensive overview of your dataset's current state and processing history.

**Key Metrics :**
1. **Total Spectra** : Total count of `*.fits.gz` files in `data/raw/`
2. **Processed Spectra** : Number of spectra already used in training (from `trained_spectra.csv`)
3. **Downloaded Plans** : Number of observation plans downloaded
4. **Top 20 Plans** : Observation plans with the highest number of spectra

**Output Information :**
- Total spectra count and processing status
- Ratio of processed vs. total spectra
- Number of observation plans downloaded
- Distribution of spectra across top 20 plans

**When to Use :**
- Before downloading new spectral batches (check available space)
- After pipeline runs (verify successful processing)
- Debugging failed runs (examine error logs)
- Planning next training sessions (estimate available data)

</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #F39C12;
    padding: 15px;
    border-radius: 8px;
    margin: 20px 0;
">

**💡 Pro Tip :**  
Run this dashboard before each major pipeline execution to ensure your dataset is in the expected state. It's particularly useful for catching duplicate downloads or incomplete processing runs.

</div>

In [ ]:
# ============================================================================
# DATASET DASHBOARD — Real-time Dataset Status Overview
# ============================================================================

visualizer.display_dataset_dashboard()

<a id="nulls"></a>

## Tool 2 : Null Features Analysis

<div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
    <a href="#toc" style="color: #2ECC71; text-decoration: none; font-size: 12px;">↩ Back to TOC</a>
</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #2ECC71;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Analyzes feature quality by generating features from a sample of raw spectra and measuring sparsity (null/zero values) across all extracted features.

**How It Works :**
1. **Spectrum Sampling** : Randomly samples N spectra from `data/raw/`
2. **Feature Generation** : Extracts features using the FeatureEngineer pipeline
3. **Sparsity Analysis** : Measures zero count, zero rate, and constant detection
4. **Statistical Summary** : Computes min, max, mean, std for each feature

**Key Parameters :**
- **n_files** : Number of spectra to sample (default: 100)
- **prominence** : Peak detection prominence parameter (default: 0.2)
- **window** : Half-window size for line analysis (default: 15 Å)
- **eps** : Threshold for "quasi-zero" values (default: 1e-12)

**Output Information :**
- DataFrame with zero_count, zero_rate, is_constant per feature
- Sorted by zero_rate (descending)
- Optional CSV/LaTeX export

**Decision Support :**
This analysis helps you decide which features to :
- **Exclude** : >80% null/zero → Remove from training
- **Impute** : 20-80% null → Consider median/mean imputation
- **Investigate** : Unexpected patterns → Check extraction code
- **Keep** : <20% null → Safe for model training

**When to Use :**
- After modifying feature engineering code
- Before model training (feature selection)
- When debugging poor model performance
- To assess feature extraction quality on new data

</div>

In [ ]:
# ============================================================================
# NULL FEATURES ANALYSIS — Feature Quality Assessment
# ============================================================================

visualizer.analyze_feature_zeros()

<a id="feature-explorer"></a>

## Tool 3 : Feature Explorer

<div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
    <a href="#toc" style="color: #2ECC71; text-decoration: none; font-size: 12px;">↩ Back to TOC</a>
</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #2ECC71;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Comprehensive exploratory data analysis (EDA) pipeline that loads feature CSV files, computes correlations, generates distributions, and trains a baseline RandomForest model for feature importance analysis.

**Analysis Pipeline :**
1. **Data Loading** : Concatenates all `features_*.csv` files from `processed/` directory
2. **Distribution Analysis** : Histograms of top variance features (max 24)
3. **Correlation Matrix** : Heatmap showing inter-feature correlations (top 30 by variance)
4. **Baseline Model** : RandomForest classifier for importance estimation
5. **Permutation Importance** : Validation-based feature importance (optional)

**Key Parameters :**
- **pattern** : Glob pattern for feature files (default: auto-detect)
- **save_dir** : Output directory for plots/CSVs (default: `logs/features/`)
- **max_hists** : Number of histogram plots (default: 24)
- **corr_top_n** : Features in correlation heatmap (default: 30)
- **rf_estimators** : Trees in baseline RF (default: 300)
- **do_permutation** : Compute permutation importance (default: True)

**Outputs :**
- CSV files: `rf_importances.csv`, `permutation_importances.csv`
- PNG plots: Histograms, correlation heatmap, importance bar charts
- Summary statistics: Missing values, distributions, top features

**When to Use :**
- After feature extraction to validate extracted features
- Before model training for feature selection
- Identifying redundant/correlated features
- Understanding feature distributions for transformations

</div>

In [ ]:
# ============================================================================
# FEATURE EXPLORER — Comprehensive EDA Pipeline
# ============================================================================
"""
This tool performs a complete exploratory analysis on extracted features:
- Loads and concatenates all feature CSV files
- Generates distribution plots and correlation matrix
- Trains baseline RandomForest for feature importance
- Computes permutation importance on validation split

All outputs (plots + CSV) are saved to logs/features/ by default.
"""

summary = visualizer.feature_explorer(
    pattern=None,           # Auto-detect feature files
    save_dir=None,          # Use default logs/features/
    max_hists=24,           # Number of histogram plots
    corr_top_n=30,          # Top features in correlation heatmap
    rf_estimators=400,      # Baseline RF trees
    random_state=42,        # Reproducibility seed
    do_permutation=True,    # Compute permutation importance
)

# Optionally re-plot feature importances bar chart with custom top N
# visualizer._plot_feature_importances_bar(
#     top_n=40,
#     save_path="logs/features/rf_importances_top40.png"
# )

<a id="subclasses"></a>

## Tool 4 : Subclass Distribution

<div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
    <a href="#toc" style="color: #2ECC71; text-decoration: none; font-size: 12px;">↩ Back to TOC</a>
</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #2ECC71;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Visualize the distribution of spectral subclasses from the catalog to assess class balance and identify rare/overrepresented types.

**How It Works :**
1. **Data Source** : Loads `master_catalog_temp.csv` from `data/catalog/`
2. **Column Detection** : Searches for 'subclass' or 'label' column
3. **Frequency Counting** : Computes value counts for all subclasses
4. **Bar Chart** : Displays top N most frequent subclasses

**Key Parameters :**
- **top_n** : Number of subclasses to display (default: 20)
- **normalize** : If True, shows percentages; if False, shows counts
- **save_path** : Optional PNG export path

**Output Information :**
- Bar chart sorted by frequency (descending)
- Can display either raw counts or percentages
- Grid lines for easier reading

**Decision Support :**
- **Balanced Dataset** : Similar counts across classes → Standard training
- **Imbalanced Dataset** : Few classes dominate → Use class weights or SMOTE
- **Rare Classes** : <50 samples → Consider merging or excluding

**When to Use :**
- Before training to choose appropriate techniques (stratification, class weights)
- After data acquisition to assess sample diversity
- When planning targeted downloads for underrepresented classes

</div>

In [ ]:
# ============================================================================
# SUBCLASS DISTRIBUTION — Class Balance Analysis
# ============================================================================

visualizer.plot_subclass_distribution(top_n=25, normalize=True)

<div style="height: 50px;"></div>

<hr style="
    border: 0;
    height: 3px;
    background: linear-gradient(90deg, transparent, #3498DB, transparent);
    border-radius: 2px;
">

<div style="height: 50px;"></div>

<div style="
    background: linear-gradient(135deg, #3498DB 0%, #2980B9 100%);
    padding: 15px 25px;
    border-radius: 10px;
    margin: 30px 0;
    text-align: center;
    border: 2px solid #3498DB;
">
    <h2 style="color: white; margin: 0; font-size: 24px; font-weight: 300; letter-spacing: 2px;">
        ◆ SPECTRAL ANALYSIS TOOLS
    </h2>
</div>

<a id="fits"></a>

## Tool 5 : FITS Header Explorer

<div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
    <a href="#toc" style="color: #3498DB; text-decoration: none; font-size: 12px;">↩ Back to TOC</a>
</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #3498DB;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Interactive tool for opening and examining FITS file headers to inspect metadata and verify data quality.

**Inspectable Metadata:**
- **Astrometry** : RA, Dec, equinox, coordinate system
- **Observational** : Date, time, exposure, seeing, airmass
- **Spectral** : Wavelength calibration, resolution, dispersion
- **Classification** : Object type, subclass, redshift
- **Quality** : SNR, flags, processing status
- **Instrument** : Telescope, spectrograph, filter

**Interface Features :**
- File browser widget for selecting `.fits.gz` files
- Structured header display (organized by category)
- Search functionality for specific keywords
- Export header to text file

**Common Use Cases :**
- **Quality Control** : Verify coordinates, dates, exposure times
- **Debugging** : Investigate "anomalous" spectra flagged by pipeline
- **Outlier Detection** : Find unusual values before batch processing
- **Metadata Extraction** : Check available fields for feature engineering

**Compatibility :**  
Supports compressed `.fits.gz` files via `astropy.io.fits` — no decompression needed.

</div>

In [ ]:
# ============================================================================
# FITS HEADER EXPLORER — Metadata Inspection Tool
# ============================================================================

visualizer.interactive_header_explorer()

<a id="tuning"></a>

## Tool 6 : Peak Detection Tuner

<div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
    <a href="#toc" style="color: #3498DB; text-decoration: none; font-size: 12px;">↩ Back to TOC</a>
</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #3498DB;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Interactive all-in-one spectrum analyzer with real-time peak detection tuning and optional AI classification.

**Interactive Controls :**
1. **Spectrum Selector** : Dropdown to choose any spectrum from `data/raw/`
2. **Prominence Slider** : Adjust peak detection sensitivity (0.01-1.0)
3. **Window Slider** : Half-window size in Å for line analysis (1-50)
4. **X-Axis Zoom** : Wavelength range slider (3500-9500 Å)
5. **Y-Axis Zoom** : Flux range slider for better visibility
6. **Model Selector** : Optional trained model for AI classification

**Visual Output :**
- **Spectrum Plot** : Normalized flux vs wavelength
- **Detected Peaks** : Highlighted with colored markers and labels
- **Known Lines** : Identified spectral lines (Hα, Hβ, CaII, Mg b, Na D)
- **Quantitative Table** : Centroids, FWHM, prominence, equivalent width
- **AI Prediction** : If model selected, displays predicted class

**Optimization Workflow :**
1. Select a representative spectrum
2. Adjust prominence to capture desired lines
3. Tune window size for optimal line detection
4. Use zoom sliders for detailed inspection
5. Record optimal parameters for batch processing

</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #F39C12;
    padding: 15px;
    border-radius: 8px;
    margin: 20px 0;
">

**💡 Best Practice :**  
Save a screenshot of your optimal settings along with the spectrum used for tuning. This creates a reference for reproducibility and parameter justification in publications.

</div>

In [ ]:
# ============================================================================
# PEAK DETECTION TUNER — Interactive Parameter Optimization
# ============================================================================

visualizer.interactive_peak_tuner()

<a id="compare-norm"></a>

## Tool 7 : Normalization Comparator

<div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
    <a href="#toc" style="color: #3498DB; text-decoration: none; font-size: 12px;">↩ Back to TOC</a>
</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #3498DB;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Visual comparison of raw vs. normalized spectra to assess the impact of continuum normalization on spectral features.

**Comparison Views:**
- **Top Panel** : Raw flux (Avant Normalisation) showing original spectral continuum
- **Bottom Panel** : Normalized flux (Après Normalisation) with removed continuum

**Key Features :**
- Side-by-side comparison of 2 spectra (default)
- Same wavelength range for direct comparison
- Automatic selection of representative spectra (random or specified)
- Export to PNG for documentation

**Parameters :**
- **sample_paths** : List of specific spectra to compare (relative paths)
- **n_samples** : Number of random spectra if paths not specified (default: 2)
- **random_state** : Seed for reproducible random selection
- **save_path** : Custom export path (default: `website/static/img/avant_apres_normalisation.png`)

**When to Use:**
- Validating normalization effectiveness
- Debugging over/under-subtraction artifacts
- Verifying that spectral lines are preserved
- Creating figures for pipeline documentation

</div>

In [ ]:
# ============================================================================
# NORMALIZATION COMPARATOR — Method Evaluation Tool
# ============================================================================

visualizer.plot_normalization_comparison()

In [ ]:
# ============================================================================
# NORMALIZATION COMPARATOR — Before/After Comparison
# ============================================================================
"""
Compares raw vs. normalized flux for 2 spectra.

Use Cases:
- Verify normalization preserves spectral lines
- Detect artifacts from over-subtraction
- Document preprocessing steps
"""

fig, axes = visualizer.plot_normalization_comparison(
    sample_paths=None,      # Auto-select 2 random spectra
    n_samples=2,            # Number of spectra to compare
    random_state=42,        # Reproducible selection
    save_path=None,         # Use default path
)

<a id="compare-spectra"></a>

## Tool 8 : Spectra Comparator

<div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
    <a href="#toc" style="color: #3498DB; text-decoration: none; font-size: 12px;">↩ Back to TOC</a>
</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #3498DB;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Interactive multi-spectrum comparison tool for identifying differences, similarities, and classification features.

**Comparison Features :**
1. **Multi-Select** : Choose multiple spectra from dropdown (Ctrl/Shift-click)
2. **Overlay Mode** : Superimpose spectra with optional Y-offset for clarity
3. **Normalization** : Toggle to compare normalized vs. raw flux
4. **Export** : Save comparison plots as PNG

**Interactive Controls :**
- Spectrum selection via multi-select dropdown
- Normalization toggle (checkbox)
- Y-offset slider for visual separation
- Export filename specification

**Analysis Capabilities :**
- Visual identification of classification features
- Line strength comparisons across spectral types
- Continuum shape analysis
- Quality assessment (SNR, artifacts)

**Common Applications :**
- **Classification** : Compare unknown spectrum to reference templates
- **Quality Check** : Identify processing artifacts
- **Feature Validation** : Verify extracted features match visual inspection
- **Documentation** : Create comparison plots for presentations

</div>

In [ ]:
# ============================================================================
# SPECTRA COMPARATOR — Multi-Spectrum Analysis Tool
# ============================================================================

visualizer.interactive_spectra_comparator()

<div style="height: 50px;"></div>

<hr style="
    border: 0;
    height: 3px;
    background: linear-gradient(90deg, transparent, #9B59B6, transparent);
    border-radius: 2px;
">

<div style="height: 50px;"></div>

<div style="
    background: linear-gradient(135deg, #9B59B6 0%, #8E44AD 100%);
    padding: 15px 25px;
    border-radius: 10px;
    margin: 30px 0;
    text-align: center;
    border: 2px solid #9B59B6;
">
    <h2 style="color: white; margin: 0; font-size: 24px; font-weight: 300; letter-spacing: 2px;">
        ◆ SPATIAL ANALYSIS TOOLS
    </h2>
</div>

<div id="sky"></div>

## Tool 9 : Sky Coverage Map

<div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
    <a href="#toc" style="color: #9B59B6; text-decoration: none; font-size: 12px;">↩ Back to TOC</a>
</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #9B59B6;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Visualize the spatial distribution of spectral observations across the celestial sphere using Mollweide projection.

**Projection Details :**
- **Type** : Mollweide (full-sky equal-area projection)
- **Coordinates** : RA/Dec (J2000)
- **RA Direction** : Right to left (astronomical convention)
- **Center** : 0h/24h RA

**Visualization Features :**
- **Point Size** : Proportional to number of spectra per observation plate
- **Point Color** : Gradient indicating spectrum density
- **Background** : Optional Milky Way overlay (if image available)
- **Coverage Statistics** : Total plates, total spectra, spatial distribution

**Data Source :**
- Automatically generates/caches position catalog from FITS headers
- One point per observation plate (not per individual spectrum)
- RA/Dec extracted from `RAOBJ`/`DECOBJ` or `RA`/`DEC` headers

**Analysis Capabilities :**
- Survey completeness assessment
- Spatial bias detection
- Observation planning (identify gaps)
- Overlap verification with other surveys

**Export :**
- Saves to `logs/sky_coverage_TIMESTAMP.png` by default
- High-resolution PNG (180 DPI) for publications

</div>

In [ ]:
# ============================================================================
# SKY COVERAGE MAP — Mollweide Spatial Distribution
# ============================================================================
"""
Generates Mollweide projection of observation coverage.

Note: First run may take longer (generates position cache).
Subsequent runs use cached catalog.
"""

fig, ax = visualizer.plot_sky_coverage(
    save_path=None  # Use default timestamp path in logs/
)

<div style="height: 50px;"></div>

<hr style="
    border: 0;
    height: 3px;
    background: linear-gradient(90deg, transparent, #E67E22, transparent);
    border-radius: 2px;
">

<div style="height: 50px;"></div>

<div style="
    background: linear-gradient(135deg, #E67E22 0%, #D35400 100%);
    padding: 15px 25px;
    border-radius: 10px;
    margin: 30px 0;
    text-align: center;
    border: 2px solid #E67E22;
">
    <h2 style="color: white; margin: 0; font-size: 24px; font-weight: 300; letter-spacing: 2px;">
        ◆ MODEL ANALYSIS TOOLS
    </h2>
</div>

<a id="inspector"></a>

## Tool 10 : Model Inspector

<div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
    <a href="#toc" style="color: #E67E22; text-decoration: none; font-size: 12px;">↩ Back to TOC</a>
</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #E67E22;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Comprehensive inspection and analysis of trained classification models saved in `data/models/` directory.

**Model Information Display :**
- **Architecture** : Model type (RF/XGBoost), hyperparameters (GridSearchCV results)
- **Training Metadata** : Date, duration, dataset size, features used
- **Performance Metrics** : Accuracy, precision, recall, F1-score (per-class)
- **Feature Configuration** : Number of features, selection method

**Performance Visualizations :**
1. **Feature Importance** : Bar chart of top features (default: top 20)
2. **Summary Statistics** : Table of per-class metrics
3. **Training History** : Logged hyperparameter search (if available)

**Interactive Interface :**
- Model selection dropdown (auto-populated from `MODELS_DIR`)
- "Analyze" button to load and inspect selected model
- "Refresh" button to rescan models directory

**Output Information :**
- Model path and file size
- Compatibility check with current code version
- Feature importance ranking
- Performance summary

**When to Use :**
- After training to verify model properties
- Before deployment to check compatibility
- Comparing multiple trained models
- Extracting feature importance for reports

</div>

In [ ]:
# ============================================================================
# MODEL INSPECTOR — Trained Model Analysis
# ============================================================================

visualizer.interactive_model_inspector()

<a id="shap"></a>

## Tool 11 : SHAP Interpretability Analysis

<div style="display: flex; justify-content: flex-end; margin-bottom: 10px;">
    <a href="#toc" style="color: #E67E22; text-decoration: none; font-size: 12px;">↩ Back to TOC</a>
</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #E67E22;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose :**  
Advanced model interpretability using SHAP (SHapley Additive exPlanations) to understand **why** the model makes specific predictions.

**Interactive Configuration :**
- **Model Selection** : Dropdown to choose trained model (`.pkl` file)
- **Sample Size** : Slider for number of spectra to analyze (100-2000)
- **Top N Features** : Number of features to show in plots (5-100)
- **Explainer Mode** : Auto, Tree, Linear, Kernel, or Permutation

**SHAP Visualizations :**

1. **Summary Bar Plot**
   - Mean absolute SHAP values per feature
   - Shows global feature importance
   - Sorted by impact magnitude

2. **Beeswarm Plot**
   - SHAP value distribution per feature
   - Color: Feature value (red=high, blue=low)
   - Position: SHAP value (impact on prediction)
   - Reveals non-linear relationships

**Explainer Types :**
- **Auto** : Automatically selects best explainer for model type
- **Tree** : Fast, optimized for RF/XGBoost (recommended)
- **Linear** : For linear models
- **Kernel** : Model-agnostic but slower
- **Permutation** : Alternative importance measure

**Output Files :**
- **CSV** : `shap_importance_TIMESTAMP.csv` with mean SHAP values
- **PNG** : Bar plot and beeswarm saved to `logs/`

**Scientific Insights :**
SHAP analysis reveals **physical mechanisms** behind classification:
- Which spectral lines drive O-star vs. B-star separation?
- How do metallicity indicators affect giant/dwarf classification?
- Are certain features redundant or synergistic?

**Computational Notes :**
- TreeExplainer (auto-selected for RF/XGBoost): ~1-10s per prediction
- KernelExplainer: Much slower, use only if necessary
- Recommended sample sizes: 500 (exploration), 2000 (publication)

</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #9B59B6;
    padding: 15px;
    border-radius: 8px;
    margin: 20px 0;
">

**💡 Research Impact :**  
SHAP analysis often leads to **publishable discoveries** about stellar physics. For example:
- "We discovered that the model relies primarily on the Mg b triplet ratio for giant/dwarf separation, consistent with surface gravity effects predicted by stellar atmosphere models."
- "SHAP analysis revealed that the Balmer decrement is the dominant temperature indicator for early-type stars."

</div>

In [ ]:
# ============================================================================
# SHAP INTERPRETABILITY — Model Explainability Analysis
# ============================================================================
"""
IMPORTANT: SHAP analysis is computationally intensive.

Recommendations:
- For exploration: 500-1000 spectra
- For publication: 1500-2000 spectra
- Expected runtime: 30 seconds to 5 minutes depending on sample size

TreeExplainer (auto-selected for RF/XGBoost) is much faster than
kernel-based explainers.
"""

visualizer.interactive_shap_explainer()

In [ ]:
# Still don't work for now

from datetime import datetime, timezone
ts = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")
out_dir = os.path.join(visualizer.paths["LOGS_DIR"], "shap"); os.makedirs(out_dir, exist_ok=True)

visualizer.plot_shap_summary_bar(top_n=25, save_path=os.path.join(out_dir, f"shap_bar_{ts}.png"))
visualizer.plot_shap_beeswarm(max_display=20, save_path=os.path.join(out_dir, f"shap_beeswarm_{ts}.png"))

<div style="height: 50px;"></div>

<hr style="
    border: 0;
    height: 4px;
    background: linear-gradient(90deg, #FF6B6B 0%, #F06595 25%, #CC5DE8 50%, #845EF7 75%, #5C7CFA 100%);
    border-radius: 2px;
    box-shadow: 0 0 10px rgba(255, 107, 107, 0.5);
">

<div style="height: 50px;"></div>


<div style="width: 100%; font-family: 'Segoe UI', Helvetica, Arial, sans-serif; box-sizing: border-box;">

<div style="
        background: linear-gradient(135deg, #FF6B6B 0%, #F06595 50%, #CC5DE8 100%);
        border-radius: 12px;
        margin: 20px 0;
        border: 1px solid rgba(255, 255, 255, 0.3);
        box-shadow: 0 10px 25px rgba(240, 101, 149, 0.4);
        box-sizing: border-box;
        overflow: hidden;
        padding: 20px;
        display: flex;
        flex-direction: column;
    ">
        


<div style="text-align: center;">
            <h2 style="
                color: white; 
                margin: 0; 
                font-size: 22px; 
                font-weight: 600; 
                letter-spacing: 3px;
                text-transform: uppercase;
                text-shadow: 0 2px 4px rgba(0,0,0,0.2);
                line-height: 1.4;
            ">
                ◆ Advanced Experimental Tools
            </h2>

<p style="
                color: rgba(255, 255, 255, 0.95); 
                margin: 8px 0 0 0; 
                font-size: 13px; 
                font-style: italic;
                font-weight: 400;
                letter-spacing: 0.5px;
            ">
                Prototype Features • Under Development • Research Grade
            </p>
        </div>

</div>

</div>

<a id="hr-diagrams"></a>

## Tool 12 (Beta): Gaia HR Diagrams

<div style="display: flex; justify-content: space-between; align-items: center; margin-bottom: 10px;">
    <span style="
        background: linear-gradient(135deg, #FF6B6B, #F06595);
        color: white;
        padding: 4px 12px;
        border-radius: 12px;
        font-size: 11px;
        font-weight: bold;
        letter-spacing: 0.5px;
    ">TESTING PHASE</span>
    <a href="#toc" style="color: #FF6B6B; text-decoration: none; font-size: 12px;">↩ Back to TOC</a>
</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #FF6B6B;
    padding: 20px;
    border-radius: 8px;
    margin: 20px 0;
">

**Purpose:**  
Generate Hertzsprung-Russell diagram variants using Gaia DR3 cross-matched photometry to visualize stellar populations and validate spectroscopic classifications.

**Diagram Types:**

1. **Color-Magnitude Diagram (CMD)**
   - X-axis: bp_rp (Gaia BP-RP color index)
   - Y-axis: M_G (Absolute G magnitude)
   - Shows: Main sequence, giants, white dwarfs
   - Quality filters: RUWE < 1.4, parallax_snr ≥ 5

2. **Reduced Proper Motion (RPM)**
   - X-axis: bp_rp (Gaia BP-RP color index)
   - Y-axis: H_G (Reduced proper motion in G)
   - Shows: Kinematic separation of stellar populations
   - Useful for: Giant/dwarf discrimination, white dwarf identification

**Key Features:**
- **Auto-Detection**: Automatically finds Gaia columns (supports corrected versions)
- **Quality Filtering**: RUWE and parallax SNR cuts for reliable astrometry
- **Subclass Coloring**: Color-coded by spectral subclass (up to 20 classes)
- **Smart Sampling**: Down-samples to 50,000-80,000 points for performance
- **Publication Ready**: 180 DPI PNG export with customizable titles

**Required Columns:**

For CMD:
- `bp_rp` or `bp_rp0` (Gaia BP-RP color)
- `M_G`, `M_G0`, `M_G_dist`, or `M_G_parallax` (Absolute G magnitude)
- `ruwe` or `is_good_ruwe` (optional: astrometric quality)
- `parallax_snr` (optional: parallax signal-to-noise)
- `subclass` or `main_class` (optional: for coloring)

For RPM:
- `bp_rp` or `bp_rp0` (Gaia BP-RP color)
- `H_G` (Reduced proper motion)
- `subclass` or `main_class` (optional: for coloring)

**Physical Interpretation:**

CMD reveals stellar evolutionary stages:
- **Main Sequence**: Diagonal band (dwarf stars, hydrogen burning)
- **Giants**: Upper right (large, cool stars)
- **White Dwarfs**: Lower left (compact, hot remnants)
- **Subgiants**: Transition region between MS and giant branch

RPM enables kinematic selection:
- **Dwarfs**: Higher H_G (larger proper motions)
- **Giants**: Lower H_G (smaller proper motions, distant)
- **Subdwarfs**: Offset from main sequence (metal-poor, higher velocities)

**Development Status:**
- Core functionality complete
- Auto-column detection working
- Quality filtering implemented
- Integration into `visualizer.py` - pending
- Interactive widget interface - planned

</div>

<div style="
    background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%);
    border-left: 5px solid #9B59B6;
    padding: 15px;
    border-radius: 8px;
    margin: 20px 0;
">

**Research Applications:**  
These diagrams enable several scientific analyses:
- **Classification Validation**: Compare spectroscopic classes to photometric locus
- **Population Studies**: Identify contamination (e.g., giants misclassified as dwarfs)
- **Distance Calibration**: Cross-check spectroscopic vs. photometric distances
- **Binary Detection**: Overluminous stars offset from main sequence

</div>

In [ ]:
# ============================================================================
# GAIA HR DIAGRAMS — Helper Functions
# ============================================================================
"""
Supporting utilities for CMD and RPM generation:
- Column auto-detection (handles multiple naming conventions)
- Quality filtering (RUWE, parallax SNR)
- UTC timestamp generation for file naming
"""

# --- HR/RPM notebook helpers ============================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import import datetime, timezone

# (Optionnel) Si tu veux sauvegarder dans data/reports/ automatiquement :
# from utils import setup_project_env, ensure_dir, utc_now_tag
# PATHS = setup_project_env(create_missing_dirs=True, verbose=False)
# REPORTS_DIR = PATHS["REPORTS_DIR"]

# Sinon, définir un dossier de sortie simple
try:
    from utils import setup_project_env, utc_now_tag
    PATHS = setup_project_env(create_missing_dirs=True, verbose=False)
    REPORTS_DIR = PATHS["REPORTS_DIR"]
except Exception:
    REPORTS_DIR = "."

def _utc_tag():
    try:
        return utc_now_tag()
    except Exception:
        return datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")

def _pick_col(df: pd.DataFrame, prefer, fallback=()):
    """Retourne la 1re colonne existante dans `prefer` puis `fallback`, ou None."""
    for c in prefer:
        if c in df.columns:
            return c
    for c in fallback:
        if c in df.columns:
            return c
    return None

def _quality_mask(df: pd.DataFrame, ruwe_col=None, snr_col=None, ruwe_max=1.4, snr_min=5):
    """Apply quality cuts for Gaia astrometry."""
    mask = pd.Series(True, index=df.index)
    if snr_col and snr_col in df.columns:
        mask &= df[snr_col].fillna(0) >= snr_min
    if ruwe_col and ruwe_col in df.columns:
        if ruwe_col == "is_good_ruwe":
            mask &= (df[ruwe_col] == 1)
        else:
            mask &= df[ruwe_col].fillna(10) < ruwe_max
    return mask

def _auto_cols_for_cmd(df: pd.DataFrame):
    """Détecte bp_rp & M_G (versions corrigées si présentes)."""
    color = _pick_col(df, ["bp_rp0", "bp_rp"])
    M_G   = _pick_col(df, ["M_G0", "M_G", "M_G_dist", "M_G_parallax"])
    ruwe  = _pick_col(df, ["ruwe", "is_good_ruwe"])
    snr   = _pick_col(df, ["parallax_snr"])
    klass = _pick_col(df, ["subclass", "main_class", "class"])
    return dict(color=color, M_G=M_G, ruwe=ruwe, snr=snr, klass=klass)

def _auto_cols_for_rpm(df: pd.DataFrame):
    """Détecte bp_rp & H_G pour RPM."""
    color = _pick_col(df, ["bp_rp0", "bp_rp"])
    H_G   = _pick_col(df, ["H_G"])
    klass = _pick_col(df, ["subclass", "main_class", "class"])
    return dict(color=color, H_G=H_G, klass=klass)

print("✓ Helper functions loaded successfully")

In [ ]:
# ============================================================================
# MAIN PLOTTING FUNCTIONS — CMD & RPM
# ============================================================================

def plot_cmd(
    df: pd.DataFrame,
    *,
    color_col: str | None = None,
    M_G_col: str | None = None,
    ruwe_col: str | None = None,
    snr_col: str | None = None,
    hue: str | None = None,
    max_points: int = 80000,
    s: float = 6.0,
    alpha: float = 0.4,
    title: str | None = None,
    save_path: str | None = None,
):
    """
    Color–Magnitude Diagram (Gaia-like). Inverse l'axe Y (magnitudes).
    - Autodétecte les colonnes si non spécifiées.
    - Applique RUWE et SNR parallaxe si fournis.
    - Peut colorer par une colonne catégorielle (ex. 'subclass').

    Retourne: (fig, ax, used_df, cols_dict)
    """
    auto = _auto_cols_for_cmd(df)
    color_col = color_col or auto["color"]
    M_G_col   = M_G_col or auto["M_G"]
    ruwe_col  = ruwe_col or auto["ruwe"]
    snr_col   = snr_col or auto["snr"]
    hue       = hue or auto["klass"]

    if not color_col or not M_G_col:
        raise ValueError(f"Colonnes nécessaires introuvables. Trouvées: {auto}")

    mask = _quality_mask(df, ruwe_col=ruwe_col, snr_col=snr_col)
    work = df[mask].copy()
    work = work[work[color_col].notna() & work[M_G_col].notna()]

    # Échantillonnage si dataset massif
    if len(work) > max_points:
        work = work.sample(max_points, random_state=42)

    fig, ax = plt.subplots(figsize=(8, 9))

    if hue and hue in work.columns:
        cats = work[hue].astype(str).fillna("NA")
        codes, uniques = pd.factorize(cats, sort=True)
        sc = ax.scatter(work[color_col], work[M_G_col], c=codes, s=s, alpha=alpha, cmap="tab20")
        # petite légende compacte (max ~20 classes pour éviter la surcharge)
        max_leg = min(len(uniques), 20)
        handles = [plt.Line2D([], [], marker="o", linestyle="", label=str(uniques[i])) for i in range(max_leg)]
        ax.legend(handles=handles, labels=[str(uniques[i]) for i in range(max_leg)],
                  title=hue, loc="upper left", bbox_to_anchor=(1.02, 1), borderaxespad=0.)
    else:
        ax.scatter(work[color_col], work[M_G_col], s=s, alpha=alpha)

    ax.invert_yaxis()
    ax.set_xlabel(f"{color_col} (mag)")
    ax.set_ylabel(f"{M_G_col} (mag)")
    if not title:
        title = f"Color–Magnitude Diagram  |  N={len(work)}"
    ax.set_title(title)

    plt.tight_layout()

    if save_path is None:
        save_path = f"{REPORTS_DIR}/cmd_{_utc_tag()}.png"
    fig.savefig(save_path, dpi=180, bbox_inches="tight")
    print(f"[CMD] Sauvegardé → {save_path}")

    return fig, ax, work, dict(color=color_col, M_G=M_G_col, ruwe=ruwe_col, snr=snr_col, hue=hue)


def plot_rpm(
    df: pd.DataFrame,
    *,
    color_col: str | None = None,
    H_G_col: str | None = None,
    hue: str | None = None,
    max_points: int = 80000,
    s: float = 6.0,
    alpha: float = 0.4,
    title: str | None = None,
    save_path: str | None = None,
):
    """
    Reduced Proper Motion diagram (H_G vs couleur). Inverse Y comme un CMD.
    Retourne: (fig, ax, used_df, cols_dict)
    """
    auto = _auto_cols_for_rpm(df)
    color_col = color_col or auto["color"]
    H_G_col   = H_G_col  or auto["H_G"]
    hue       = hue or auto["klass"]

    if not color_col or not H_G_col:
        raise ValueError(f"Colonnes nécessaires introuvables. Trouvées: {auto}")

    work = df.copy()
    work = work[work[color_col].notna() & work[H_G_col].notna()]
    if len(work) > max_points:
        work = work.sample(max_points, random_state=42)

    fig, ax = plt.subplots(figsize=(8, 9))

    if hue and hue in work.columns:
        cats = work[hue].astype(str).fillna("NA")
        codes, uniques = pd.factorize(cats, sort=True)
        sc = ax.scatter(work[color_col], work[H_G_col], c=codes, s=s, alpha=alpha, cmap="tab20")
        max_leg = min(len(uniques), 20)
        handles = [plt.Line2D([], [], marker="o", linestyle="", label=str(uniques[i])) for i in range(max_leg)]
        ax.legend(handles=handles, labels=[str(uniques[i]) for i in range(max_leg)],
                  title=hue, loc="upper left", bbox_to_anchor=(1.02, 1), borderaxespad=0.)
    else:
        ax.scatter(work[color_col], work[H_G_col], s=s, alpha=alpha)

    ax.invert_yaxis()
    ax.set_xlabel(f"{color_col} (mag)")
    ax.set_ylabel(f"{H_G_col} (mag)")
    if not title:
        title = f"Reduced Proper Motion Diagram  |  N={len(work)}"
    ax.set_title(title)

    plt.tight_layout()

    if save_path is None:
        save_path = f"{REPORTS_DIR}/rpm_{_utc_tag()}.png"
    fig.savefig(save_path, dpi=180, bbox_inches="tight")
    print(f"[RPM] Sauvegardé → {save_path}")

    return fig, ax, work, dict(color=color_col, H_G=H_G_col, hue=hue)

print("✓ CMD and RPM plotting functions ready")

In [ ]:
# Chemin vers ton fichier (met le tien ici)
csv_path = "../data/processed/features_20250825T235455Z.csv"  # ou ton dernier features_*.csv
df = pd.read_csv(csv_path)

# HR (CMD) — couleur par sous-classe si dispo
_, _, df_cmd, cols_cmd = plot_cmd(
    df,
    hue="subclass",           # auto-retombe si la colonne n’existe pas
    max_points=50000,         # échantillonnage pour rester fluide
)
print("Colonnes CMD utilisées:", cols_cmd)

# RPM — couleur par sous-classe si dispo
_, _, df_rpm, cols_rpm = plot_rpm(
    df,
    hue="subclass",
    max_points=50000,
)
print("Colonnes RPM utilisées:", cols_rpm)


In [ ]:
# ============================================================================
# USAGE EXAMPLE — Generate CMD & RPM Diagrams
# ============================================================================
"""
This cell demonstrates how to use the HR diagram functions.

Prerequisites:
1. Feature CSV with Gaia cross-matched photometry
2. Required columns: bp_rp (or bp_rp0), M_G (or variants), H_G
3. Optional: ruwe, parallax_snr, subclass for filtering/coloring

Output:
- Two publication-ready plots (CMD and RPM)
- Saved as PNG in data/reports/ (or current directory)
- Returns figure objects for further customization
"""

# Path to your latest feature file (update this!)
csv_path = "../data/processed/features_20250825T235455Z.csv"  # ← CHANGE THIS

# Load data
print(f"Loading features from: {csv_path}")
df = pd.read_csv(csv_path)
print(f"  → Loaded {len(df):,} spectra")
print(f"  → Columns: {list(df.columns[:10])}...")

print("\n" + "="*70)
print("GENERATING COLOR-MAGNITUDE DIAGRAM (CMD)")
print("="*70)

# Generate CMD — color by spectral subclass if available
fig_cmd, ax_cmd, df_cmd, cols_cmd = plot_cmd(
    df,
    hue="subclass",           # Color by subclass (auto-fallback if missing)
    max_points=50000,         # Subsample for performance
    ruwe_col="ruwe",          # Apply RUWE quality cut
    snr_col="parallax_snr",   # Apply parallax SNR cut
)

print(f"\n→ Columns used: {cols_cmd}")
print(f"→ Points plotted: {len(df_cmd):,} (after quality cuts + sampling)")

print("\n" + "="*70)
print("GENERATING REDUCED PROPER MOTION DIAGRAM (RPM)")
print("="*70)

# Generate RPM — color by spectral subclass if available
fig_rpm, ax_rpm, df_rpm, cols_rpm = plot_rpm(
    df,
    hue="subclass",           # Color by subclass (auto-fallback if missing)
    max_points=50000,         # Subsample for performance
)

print(f"\n→ Columns used: {cols_rpm}")
print(f"→ Points plotted: {len(df_rpm):,}")

print("\n" + "="*70)
print("✓ DIAGRAMS GENERATED SUCCESSFULLY")
print("="*70)
print("\nCheck your data/reports/ directory for PNG outputs!")

<div style="height: 50px;"></div>

<hr style="
    border: 0;
    height: 3px;
    background: linear-gradient(90deg, transparent, #667eea, transparent);
    border-radius: 2px;
">

<div style="height: 50px;"></div>

<div style="
    background: linear-gradient(135deg, #667eea 0%, #764ba2 50%, #f093fb 100%);
    border-radius: 12px;
    padding: 30px;
    text-align: center;
    margin-top: 50px;
    border: 2px solid #667eea;
">
    <h2 style="color: white; margin: 0; font-size: 24px; font-weight: 300; letter-spacing: 2px;">
        Analysis Tools Ready
    </h2>
    <p style="color: #f0f0f0; margin-top: 15px; font-size: 14px; line-height: 1.8;">
        All 11 diagnostic and visualization tools are now available.<br>
        Use these tools to explore, debug, and understand your spectral dataset and trained models.<br>
        <strong>For model training:</strong> Return to <a href="./00_master_pipeline.ipynb" style="color: white; text-decoration: none; border-bottom: 1px solid white;">00_master_pipeline.ipynb</a>
    </p>
    <div style="margin-top: 25px; padding-top: 20px; border-top: 1px solid rgba(255, 255, 255, 0.3);">
        <p style="color: #f0f0f0; font-size: 12px; margin: 0;">
            <strong>AstroSpectro</strong> — Automated Stellar Classification Pipeline<br>
            GitHub: <a href="https://github.com/PhD-Brown/AstroSpectro" style="color: white; text-decoration: none;">PhD-Brown/AstroSpectro</a> •
            Docs: <a href="https://phd-brown.github.io/AstroSpectro/" style="color: white; text-decoration: none;">phd-brown.github.io/AstroSpectro</a>
        </p>
    </div>
</div>